# Phase 3: Model Deployment 

In the final phase of the assignment, we focus on deploying the optimized model using Google Cloud's Vertex AI. This phase leverages the `gcloud` command-line interface (CLI) for deploying and managing machine learning models on the cloud platform.

## Rationale for Using gcloud CLI

Vertex AI offers various interfaces for interaction, including the UI (Cloud Console), the `gcloud` CLI, and SDKs for languages such as Python and Java. While the UI is convenient for exploratory and manual tasks, it is less suited for production environments due to its less programmable nature. On the other hand, SDKs provide deep integration within codebases but require adjustments whenever deployment specifications change. 

The `gcloud` CLI strikes a perfect balance by offering several advantages for production-grade deployments:
1. **Scriptability and Automation**: The CLI allows for scripting complex operations which can be automated without manual intervention. This is crucial for production environments where deployments need to be reproducible and consistent.
2. **Parameterization**: CLI commands can be parameterized easily within scripts. This allows for changing deployment parameters such as model names, instance types, regions, and other configurations without altering the core logic of the deployment scripts.
3. **Flexibility**: It provides the flexibility needed in dynamic production environments. Changes can be rolled out quickly by adjusting the parameters in the CLI commands.
4. **No Code-Level Changes Required**: Using the CLI reduces the need for frequent code changes. Deployment scenarios can be handled by script adjustments, minimizing the potential for bugs and reducing development overhead.

## Deployment Strategy Using gcloud CLI

The deployment process is streamlined into a sequence of `gcloud` commands that handle tasks such as:
- Uploading the model to Vertex AI.
- Creating an endpoint for the model.
- Deploying the model to the endpoint with specific machine types and scaling configurations.

Each of these steps can be executed via a single command, with parameters controlled through command-line arguments or environment variables, ensuring that the process is both flexible and easily repeatable.

By utilizing the `gcloud` CLI for deploying models on Vertex AI, we achieve a high degree of automation and flexibility, crucial for scaling machine learning operations in production environments. This approach helps maintain a clear separation between deployment logistics and model development, leading to more robust and maintainable workflows.

In summary, the `gcloud` CLI is an effective tool for managing Vertex AI operations, advisable for scenarios demanding regular updates or deployments without the need for direct code manipulation, suitable for continuous integration/continuous deployment (CI/CD) pipelines.

After completing the hyperparameter tuning phase and identifying the best performing model based on the AUC-PR metric, we must prepare the model for deployment on Vertex AI. Vertex AI requires models to be in a specific format (.bst for XGBoost) for optimal compatibility with its managed services. Thus, converting our model from JSON to BST format becomes necessary before proceeding to model registration and deployment.

### Step 1: Model Format Conversion

The model trained and saved in JSON format needs to be converted to the BST format, which is the native binary format used by XGBoost, and is preferred for deployment on Vertex AI due to its efficiency in load times and compatibility. The conversion involves renaming the model file from `.json` to `.bst`. This is done using the `gsutil mv` command, which moves and renames the model file within Google Cloud Storage. Here's how this is accomplished:

In [1]:
!gsutil mv gs://telco-churn-prediction/model/xgboost_model17128808474388263.json \
    gs://telco-churn-prediction/model/model.bst



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Copying gs://telco-churn-prediction/model/xgboost_model17128808474388263.json [Content-Type=application/json]...
Removing gs://telco-churn-prediction/model/xgboost_model17128808474388263.json...

Operation completed over 1 objects/341.5 KiB.                                    


This command effectively renames 'xgboost_model17128808474388263.json' to 'model.bst', preparing it for deployment.

### Step 2: Uploading the Model to Vertex AI

Once the model is in the correct format, the next step is to register it with Vertex AI's Model Registry. This allows the model to be managed, versioned, and deployed systematically. The model is uploaded using the `gcloud ai models upload` command, which specifies necessary parameters such as the region, display name, the container image for running the model, and the location of the model artifacts. Here is the command used:

In [2]:
!gcloud ai models upload \
  --region=asia-south1 \
  --display-name=churn-prediction-xgboost-best-model \
  --container-image-uri=asia-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest \
  --artifact-uri=gs://telco-churn-prediction/model/

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
Waiting for operation [7564476171866341376]...done.                            


#### Key Parameters Explained:
- **--region**: Specifies the Google Cloud region where the model will be hosted. This should match the region where the endpoints will be deployed.
- **--display-name**: A user-friendly name for the model in the Vertex AI dashboard.
- **--container-image-uri**: This parameter points to a container image that is used to serve the model. For XGBoost models, Vertex AI offers optimized container images.
- **--artifact-uri**: The URI where the model's artifacts are stored. It should point to the directory containing the `.bst` file.

### Step 3: Model Listing
To start the deployment process, begin by listing all the models available in the specified region under your Google Cloud project. This provides a comprehensive view of all models, including their unique identifiers and display names. Listing the models is crucial to ensure accurate model selection for deployment.

In [13]:
!gcloud ai models list --region=asia-south1

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
MODEL_ID                                          DISPLAY_NAME
3839608953403080704                               churn-prediction-xgboost-best-model
5122404771482828800                               xgboost-best-model-hpo-1-v2
7821257216928776192                               universal_model_low_features
mistralai_mistral-7b-instruct-v0_1-1704100088152  mistralai_mistral-7b-instruct-v0_1-1704100088152
4554458637224902656                               utech-universal-model
5520058544830808064                               current_phase_c
192300185651511296                                current_phase_b
4803986204078899200                               current_phase_a
3214215535617114112                               total_active_energy
7825901554044502016                               total_active_power
2061294031010267136                               voltage_a_n
6672980049437655040                               voltage_b_

### Step 4: Endpoint Creation
Create a new endpoint in Vertex AI to serve predictions. An endpoint acts as a consolidated gateway through which different versions of your models can be accessed. The creation of endpoints is pivotal as it defines the environment and attributes such as the region, which plays a vital role in latency and accessibility for prediction requests.

In [3]:
!gcloud ai endpoints create \
  --region=asia-south1 \
  --display-name=churn-prediction-xgboost-best-model

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
Waiting for operation [8220875817555591168]...done.                            
Created Vertex AI endpoint: projects/777232604101/locations/asia-south1/endpoints/3538624242369167360.


### Step 5: Listing Endpoints
After creating the endpoint, list the available endpoints in the region to confirm the successful creation and to check for details such as endpoint IDs, which are necessary for deploying models to these endpoints.

In [4]:
!gcloud ai endpoints list --region=asia-south1

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
3538624242369167360  churn-prediction-xgboost-best-model
5844150592234061824  2024-01-01-utech-energy


### Step 6: Model Deployment to Endpoint
Deploy the specific model to the recently created endpoint. This process includes specifying the model by its ID, defining display names for organizational clarity, and setting operational parameters such as the number of replicas. These configurations are essential for handling load and ensuring reliability.

In [14]:
!gcloud ai endpoints deploy-model 3538624242369167360 \
  --region=asia-south1  \
  --model=3839608953403080704 \
  --display-name=churn-prediction-xgboost \
  --min-replica-count=1 \
  --max-replica-count=2 \
  --traffic-split=0=100

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
Waiting for operation [7288630694689898496]...done.                            
Deployed a model to the endpoint 3538624242369167360. Id of the deployed model: 3579789957713100800.


### Step 7: Endpoint Configuration and Traffic Management
Post-deployment, confirm the configuration by describing the endpoint. This description should include detailed configurations like the number of replicas, the machine type used, and other deployment-specific details. It should also show the traffic split, which is primarily used to manage multiple model versions under the same endpoint, directing a percentage of incoming traffic to different model variants.

Each of these steps plays a crucial role in the deployment lifecycle within Vertex AI, designed to streamline and simplify the process of getting models from training to serving. This approach ensures that the models are not only deployed efficiently but are also scalable and manageable in a production environment.

In [15]:
!gcloud ai endpoints describe 3538624242369167360 --region=asia-south1

Using endpoint [https://asia-south1-aiplatform.googleapis.com/]
createTime: '2024-04-12T11:09:27.874162Z'
deployedModels:
- createTime: '2024-04-12T11:36:33.249975Z'
  dedicatedResources:
    machineSpec:
      machineType: n1-standard-2
    maxReplicaCount: 2
    minReplicaCount: 1
  displayName: churn-prediction-xgboost
  id: '3579789957713100800'
  model: projects/777232604101/locations/asia-south1/models/3839608953403080704
  modelVersionId: '1'
displayName: churn-prediction-xgboost-best-model
etag: AMEw9yO7Wku4sD8JyUEwG9T9CSGH8wspStZe6UJx26oGuPo-81i-AlBcm8J0bJANQT2q
name: projects/777232604101/locations/asia-south1/endpoints/3538624242369167360
trafficSplit:
  '3579789957713100800': 100
updateTime: '2024-04-12T11:46:16.994965Z'


## Process for Making Real-Time Predictions

The process outlined for making real-time predictions leverages the capabilities of Google Cloud’s Vertex AI to deploy a machine learning model that forecasts customer churn. The predictions are derived from a preprocessed dataset, where data transformation and feature engineering are critical components ensuring data compatibility with the model’s input requirements.

### Data Preparation and Preprocessing
- **Data Loading and Initial Processing:** The dataset is loaded into a pandas DataFrame. Key transformations include converting 'TotalCharges' from a string to numeric and handling missing values.
- **Feature Engineering:** New features are derived, such as 'AverageMonthlyCharges', and 'tenure' is categorized into groups. These steps are aimed at enriching the model's input to capture more complex patterns.
- **Encoding and Scaling:** Categorical variables are encoded using both label encoding and one-hot encoding to transform them into a machine-readable format. Numerical features are scaled to ensure model sensitivity is uniform across variables.

In [1]:
import pandas as pd
import numpy as np
import json

import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from google.cloud import aiplatform
from google.oauth2 import service_account

In [2]:
# Load the dataset
df = pd.read_csv('../data/telco_customer_churn_data.csv')
df_original = df.copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])
# Create 'AverageMonthlyCharges' by dividing 'TotalCharges' by 'tenure'
# Handle cases where tenure is zero: AverageMonthlyCharges should be the same as MonthlyCharges
df['AverageMonthlyCharges'] = np.where(df['tenure'] != 0, df['TotalCharges'] / df['tenure'], df['MonthlyCharges'])

# Categorize 'tenure' into different groups
df['TenureGroups'] = pd.cut(df['tenure'], bins=[0, 12, 24, 48, 60, np.inf], labels=['0-1 year', '1-2 years', '2-4 years', '4-5 years', '5+ years'])

# List of binary categorical columns to encode as 0 and 1
binary_columns = ["gender", "Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]

# Encode binary categorical variables using LabelEncoder
label_encoder = LabelEncoder()
for col in binary_columns:
    df[col] = label_encoder.fit_transform(df[col])

# List of nominal columns to apply one-hot encoding
nominal_columns = [
    "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup",
    "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies",
    "Contract", "PaymentMethod", "TenureGroups"
]

# Apply one-hot encoding using get_dummies in pandas
df = pd.get_dummies(df, columns=nominal_columns, drop_first=False, dtype=int)

# List of numerical columns to scale
numerical_columns = ["AverageMonthlyCharges", "tenure", "MonthlyCharges", "TotalCharges"]

# Initialize MinMaxScaler to scale numerical columns to be between 0 and 1
scaler = MinMaxScaler()

# Apply scaler separately for each column
for col in numerical_columns:
    df[col] = df[col].astype(float)  # Ensure data type is float for scaling
    df[col] = scaler.fit_transform(df[[col]])

### Serialization and Prediction
- **Data Serialization:** A subset of the dataset intended for demonstration purposes is selected and serialized into JSON format. This serialized data represents the model input for making predictions.
- **Invoking the Prediction:** The serialized input is fed to a model hosted on a Vertex AI endpoint using the `gcloud ai endpoints predict` command. This setup helps in making real-time predictions by providing an automated way to query the model.

In [3]:
# Select a subset for demonstration (e.g., 100 rows)
demo_df = df.sample(n=10, random_state=42)

# Keep customerID for later reference and drop it from the data sent to the model
customer_ids = demo_df['customerID']
labels = demo_df['Churn']
X_demo = demo_df.drop(columns=['customerID', 'Churn'])

# Convert the DataFrame to a list of lists for prediction
input_data = X_demo.values.tolist()

In [4]:
demo_df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'Churn', 'AverageMonthlyCharges', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank tran

In [5]:
# Prepare the JSON payload
payload = {"instances": input_data}

# Serialize and save payload to JSON file
json_file_path = 'payload.json'
with open(json_file_path, 'w') as json_file:
    json.dump(payload, json_file, indent=4)

In [6]:
!gcloud ai endpoints predict 3538624242369167360 \
  --region=asia-south1 \
  --json-request=payload.json >> output.txt

Using endpoint [https://asia-south1-prediction-aiplatform.googleapis.com/]


### Post-Prediction Processing
- **Interpreting Results:** Predictions obtained from the model are specific scores indicating the likelihood of churn. These scores are converted into binary labels based on a threshold, typically 0.5, distinguishing between 'Churn' and 'Not Churn'.
- **Mapping Predictions to Customer IDs:** For practical deployment and review purposes, each prediction is associated back to a customer ID, providing a straightforward reference to actual data.

### Results Presentation
- **Output Delivery:** The final step involves presenting the predicted results alongside actual labels for each customer. This comparative format is crucial for validating model performance and is essential for operational transparency.

This sequential approach to making real-time predictions systematically addresses various stages from data handling to result presentation, ensuring the model deployed in Vertex AI is utilized effectively. Employing Google Cloud's infrastructure facilitates robust scalability and real-time processing capabilities necessary for responsive machine learning applications. Through this documented framework, stakeholders can replicate or modify the prediction process according to specific business requirements or operational contexts.

In [7]:
# Load and parse the predictions from the file
with open('output.txt', 'r') as file:
    predictions = json.load(file)

# Apply a threshold of 0.5 to convert predictions to binary labels
binary_labels = [1 if float(pred) >= 0.5 else 0 for pred in predictions]

# Map binary labels to "Churn" or "Not Churn"
churn_labels = ['Churn' if label == 1 else 'Not Churn' for label in binary_labels]

# Print results for each customer ID
for customer_id, churn_label, label in zip(customer_ids, churn_labels, labels):
    print(f'Customer ID: {customer_id}, Prediction: {churn_label}, Actul: {churn_label}')


Customer ID: 6614-YWYSC, Prediction: Not Churn, Actul: Not Churn
Customer ID: 9546-KDTRB, Prediction: Not Churn, Actul: Not Churn
Customer ID: 0871-URUWO, Prediction: Churn, Actul: Churn
Customer ID: 5151-HQRDG, Prediction: Not Churn, Actul: Not Churn
Customer ID: 6624-JDRDS, Prediction: Not Churn, Actul: Not Churn
Customer ID: 3082-VQXNH, Prediction: Churn, Actul: Churn
Customer ID: 1309-XGFSN, Prediction: Not Churn, Actul: Not Churn
Customer ID: 9402-ORRAH, Prediction: Churn, Actul: Churn
Customer ID: 8663-UPDGF, Prediction: Not Churn, Actul: Not Churn
Customer ID: 0455-ENTCR, Prediction: Not Churn, Actul: Not Churn


# Generating Personalized Retention Incentives with OpenAI Assistants

To enhance customer retention efforts, this process leverages the OpenAI API to generate personalized incentives for customers predicted to churn. The approach combines advanced machine learning predictions with natural language generation capabilities of GPT-4 to produce tailored, concise, and engaging messages that aim to improve customer retention rates.

### **Step 1: Assistant Creation**
An assistant is created using the OpenAI API, specifically designed as a 'Customer Retention Specialist'. This assistant is programmed with instructions to generate short, single-sentence retention incentives based on detailed customer profiles and churn predictions.

In [8]:
from openai import OpenAI
client = OpenAI()
  
assistant = client.beta.assistants.create(
  name="Customer Retention Specialist",
  instructions="""You work as a Customer Retention Specialist at a large telecom company. Your responsibility is to generate personalized retention incentives based on churn predictions from your model when customer details are provided. Ensure it is very concise, limited to a maximum of single sentences, and ready to share directly with the customer.""",
  model="gpt-4-turbo-preview",
)

### **Step 2: Thread Initialization**
A thread is initiated to handle interactions with the assistant. This thread acts as a container for the conversation, maintaining context and ensuring that the assistant's responses remain coherent and relevant to the provided customer data.

In [9]:
thread = client.beta.threads.create()

### **Step 3: Descriptive Prompt Generation**
For each customer predicted to churn, a descriptive prompt is constructed using their profile data. This prompt includes demographic details, service usage, and payment methods, formatted into a concise description. This ensures that the assistant has all the necessary context to generate a personalized retention message.

### **Step 4: Messaging and Interaction**
A message containing the customer's profile along with churn predictions is added to the thread. This message serves as the input to the assistant, prompting it to generate a retention incentive.

### **Step 5: Execution and Response Handling**
The thread is executed to generate a response from the assistant. Once the run completes, the responses are retrieved, and if successful, the assistant’s suggestions for retention incentives are extracted. These suggestions are then ready to be reviewed and potentially communicated to the customers.

In [11]:
# Iterate through each Customer ID
customer_ids = ['0871-URUWO', '3082-VQXNH', '9402-ORRAH']  # Example customer IDs


for customer_id in customer_ids:
    # Retrieve the customer row from the original DataFrame
    customer_row = df_original[df_original['customerID'] == customer_id].iloc[0]

    # Construct a descriptive sentence containing customer details
    description = (
        f"Customer ID {customer_row['customerID']}:\\n"
        f"- Gender: {customer_row['gender'].lower()}, "
        f"{'Senior' if customer_row['SeniorCitizen'] == 1 else 'Non-senior'} citizen\\n"
        f"- Relationship Status: {'Partnered' if customer_row['Partner'] == 'Yes' else 'Single'}; "
        f"{'Has dependents' if customer_row['Dependents'] == 'Yes' else 'No dependents'}\\n"
        f"- Service Duration: {customer_row['tenure']} months; Uses {customer_row['InternetService']} internet service\\n"
        f"- Current Plan: {customer_row['Contract']} contract; Paying ${customer_row['MonthlyCharges']:.2f} monthly\\n"
        f"- Billing Method: {customer_row['PaymentMethod']}, "
        f"{'Opted for paperless billing' if customer_row['PaperlessBilling'] == 'Yes' else 'Paper billing'}"
    )

    # Log the customer details for clarity
    print(f"--- Client Details for Customer ID: {customer_id} ---\n\n{description}\n\n")

    # Add the customer message to the thread for generating the incentive
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=description
    )

    # Generate the incentive response from the assistant
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Process and display the incentive plan
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        incentive_plan = json.loads(messages.json())['data'][0]['content'][0]['text']['value']
        print(f"*** Incentive Plan for Customer ID {customer_id} ***\\n{incentive_plan}\n\n")
    else:
        print(f"Failed to generate incentive for Customer ID {customer_id}, Status: {run.status}\n\n")


--- Client Details for Customer ID: 0871-URUWO ---

Customer ID 0871-URUWO:\n- Gender: male, Non-senior citizen\n- Relationship Status: Partnered; No dependents\n- Service Duration: 13 months; Uses Fiber optic internet service\n- Current Plan: Month-to-month contract; Paying $102.25 monthly\n- Billing Method: Credit card (automatic), Opted for paperless billing


*** Incentive Plan for Customer ID 0871-URUWO ***\n"We value your choice of our premium services and would like to offer you a special 10% discount on your monthly bill, bringing it down to $92.03 for the next 4 months as a token of our appreciation."


--- Client Details for Customer ID: 3082-VQXNH ---

Customer ID 3082-VQXNH:\n- Gender: male, Non-senior citizen\n- Relationship Status: Partnered; No dependents\n- Service Duration: 3 months; Uses DSL internet service\n- Current Plan: Month-to-month contract; Paying $29.80 monthly\n- Billing Method: Credit card (automatic), Paper billing


*** Incentive Plan for Customer ID 308


## Utilization Strategy
This integration of GPT-4 with predictive churn models allows for a dynamic and responsive approach to customer retention. By automating the generation of personalized incentives, the system not only scales the retention efforts but also ensures relevancy and personalization that could significantly enhance customer satisfaction and loyalty.

The use of AI-driven tools facilitates an innovative approach to customer retention, combining the predictive power of machine learning with the nuanced understanding of human language provided by natural language processing models. This strategy underscores a commitment to leveraging cutting-edge technology to improve business outcomes and customer relationships.